In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.api import Logit, GLM
from statsmodels.tools import add_constant
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tools import condense_upper_triangular, find_highest_correlations

In [3]:
data = pd.read_csv('data/Default.csv')
data.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [5]:
data_with_dummies = pd.get_dummies(data, drop_first=True, dtype=int)
data_with_dummies.head()

,balance,income,default_Yes,student_Yes
0,729.526495,44361.625074,0,0
1,817.180407,12106.134700,0,1
2,1073.549164,31767.138947,0,0
3,529.250605,35704.493935,0,0
4,785.655883,38463.495879,0,0


In [6]:
data_with_dummies.describe()

,balance,income,default_Yes,student_Yes
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,835.374886,33516.981876,0.033300,0.294400
std,483.714985,13336.639563,0.179428,0.455795
min,0.000000,771.967729,0.000000,0.000000
25%,481.731105,21340.462903,0.000000,0.000000
50%,823.636973,34552.644802,0.000000,0.000000
75%,1166.308386,43807.729272,0.000000,1.000000
max,2654.322576,73554.233495,1.000000,1.000000


In [8]:
X = add_constant(data_with_dummies[['balance', 'income']])
y = data_with_dummies['default_Yes']
log_reg = Logit(y, X).fit()
log_reg.summary()

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            default_Yes   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Mon, 01 Jul 2024   Pseudo R-squ.:                  0.4594
Time:                        08:26:01   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0.000     24.835      0.000       0.005       0.006
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [14]:
log_reg.predict(X)
accuracy = np.mean((log_reg.predict(X) > 0.5) == y) # sum of correct over n observations
accuracy

0.9737

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((8000, 3), (2000, 3))

In [12]:
log_reg_1 = Logit(y_train, X_train).fit()
log_reg_1.summary()

Optimization terminated successfully.
         Current function value: 0.076230
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            default_Yes   No. Observations:                 8000
Model:                          Logit   Df Residuals:                     7997
Method:                           MLE   Df Model:                            2
Date:                Mon, 01 Jul 2024   Pseudo R-squ.:                  0.4743
Time:                        08:30:51   Log-Likelihood:                -609.84
converged:                       True   LL-Null:                       -1160.2
Covariance Type:            nonrobust   LLR p-value:                9.963e-240
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.8297      0.506    -23.387      0.000     -12.821     -10.838
balance        0.0058      0.000     22.038      0.000       0.005       0.006
income      2.075e-05   5.64e-06      3.682      0.000    9.71e-06    3.18e-05
==============================================================================

Possibly complete quasi-separation: A fraction 0.16 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [13]:
y_preds = log_reg_1.predict(X_test)
accuracy = np.mean((y_preds > 0.5) == y_test)
accuracy

0.9695

In [18]:
glm = GLM(y_train, X_train, family=sm.families.Binomial()).fit()
glm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            default_Yes   No. Observations:                 8000
Model:                            GLM   Df Residuals:                     7997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -609.84
Date:                Mon, 01 Jul 2024   Deviance:                       1219.7
Time:                        08:40:32   Pearson chi2:                 6.64e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1285
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.8297      0.506    -23.387      0.000     -12.821     -10.838
balance        0.0058      0.000     22.038      0.000       0.005       0.006
income      2.075e-05   5.64e-06      3.682      0.000    9.71e-06    3.18e-05
==============================================================================
"""

In [23]:
def boot_fn(data, index):
    # Extract the bootstrap sample
    boot_sample = data.iloc[index]

    boot_model = GLM(boot_sample['default_Yes'], add_constant(boot_sample[['income', 'balance']]), family=sm.families.Binomial()).fit()
    # Fit the logistic regression model on the bootstrap sample
    
    # Return the coefficient estimates for income and balance
    return boot_model.params[['income', 'balance']]

# Number of bootstrap samples
n_bootstrap = 1000

# Collect bootstrap estimates
bootstrap_estimates = np.zeros((n_bootstrap, 2))

for i in range(n_bootstrap):
    # Generate a bootstrap sample index
    sample_index = resample(range(len(data_with_dummies)), replace=True, n_samples=len(data_with_dummies))
    
    # Get the bootstrap estimates
    bootstrap_estimates[i, :] = boot_fn(data_with_dummies, sample_index)

# Compute standard errors of bootstrap estimates
bootstrap_se = bootstrap_estimates.std(axis=0)
print('Bootstrap Standard Errors:', bootstrap_se)

Bootstrap Standard Errors: [4.90761648e-06 2.28339137e-04]
